<a href="https://colab.research.google.com/github/jiyeol9081/jiyeol9081.github.io/blob/main/Segmenation_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Package

In [1]:
!pip install tensorflow_datasets

# Config

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

# Dataset

In [ ]:
def process(data):
    image = tf.image.resize(data['image'], (256,256))
    label = data['segmentation_mask'][...,0]-1
    label = tf.one_hot(label, 3)
    label = tf.image.resize(label,(256,256))
    return image,label

datasets, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)
train_set = datasets['train']
test_set = datasets['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
train_set = train_set.map(process).batch(32)
test_set = test_set.map(process).batch(32)

# Visualize

In [ ]:
# Visualize 1
test_image = []
test_lable = []
for images, labels in train_set.take(1):
    for image, label in zip(images, labels):
        print(type(image))
        test_image = image
        test_label = label
        plt.imshow(image.numpy().astype("uint8"))
        plt.axis("off")
        plt.show()
        break

In [ ]:
# Visualize 2
image , label =train_set.take(1).get_single_element()
plt.imshow(image[0].numpy().astype("uint8"))
#print(label)
print(label.shape)
plt.imshow(label[0])
plt.show()

# Loss function && Metric

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [ ]:
import tensorflow as tf

class WeightedBCELoss(tf.keras.losses.Loss):
    def __init__(self, weight_positive, name="weighted_bce_loss"):
        super(WeightedBCELoss, self).__init__(name=name)
        self.weight_positive = weight_positive

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)

        bce = self.weight_positive * (-1) * y_true * tf.math.log(tf.maximum(y_pred, 1e-8)) \
              - (1 - y_true) * tf.math.log(tf.maximum(1 - y_pred, 1e-8))

        print("WBCE shape:", bce.shape)
        print("y_true shape:", y_true.shape)
        print("Weighted_positive", self.weight_positive)

        return tf.reduce_mean(bce)

# 사용 예제:
# loss_fn = WeightedBCELoss(weight_positive=3.0)
# model.compile(optimizer='adam', loss=loss_fn)


# Model

In [ ]:
# baseline CNN
model = models.Sequential([

    layers.Conv2D(16, kernel_size=(3,3), activation='relu', padding='same', input_shape=(256,256,3), kernel_initializer='he_normal'),
    layers.Conv2D(16, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(16, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.MaxPool2D(2,2),

    layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.MaxPool2D(2,2),

    layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),

    layers.UpSampling2D(),
    layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),

    layers.UpSampling2D(),
    layers.Conv2D(16, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(16, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),
    layers.Conv2D(16, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'),

    layers.Conv2D(3, kernel_size=(3,3), padding='same', activation='softmax'),

])

model.summary()

In [ ]:
# Unet
def create_unet_model():
    inputs = Input(shape=(256, 256, 3))

    # Encoder
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    # Decoder
    up4 = UpSampling2D()(pool3)
    merge4 = concatenate([conv3, up4], axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv4)

    up5 = UpSampling2D()(conv4)
    merge5 = concatenate([conv2, up5], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    up6 = UpSampling2D()(conv5)
    merge6 = concatenate([conv1, up6], axis=3)
    conv6 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    output = Conv2D(3, (3, 3), activation='softmax', padding='same')(conv6)

    model = Model(inputs=inputs, outputs=output)
    return model

model = create_unet_model()
model.summary()


In [ ]:
# U-net + Large Kernel in Initial Layer
def Large_kernel_model():
    inputs = Input(shape=(256, 256, 3))

    # Encoder
    conv1 = Conv2D(16, (31, 31), activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    # Decoder
    up4 = UpSampling2D()(pool3)
    merge4 = concatenate([conv3, up4], axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv4)

    up5 = UpSampling2D()(conv4)
    merge5 = concatenate([conv2, up5], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    up6 = UpSampling2D()(conv5)
    merge6 = concatenate([conv1, up6], axis=3)
    conv6 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    output = Conv2D(3, (3, 3), activation='softmax', padding='same')(conv6)

    model = Model(inputs=inputs, outputs=output)
    return model

# Create and summarize the model
model = Large_kernel_model()
model.summary()


In [ ]:
# HRnet
def create_HRnet_model():
    inputs = Input(shape=(256, 256, 3))

    # Encoder
    conv1 = Conv2D(16, (31, 31), activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    # Decoder
    up4 = UpSampling2D()(pool3)
    merge4 = concatenate([conv3, up4], axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv4)

    up5 = UpSampling2D()(conv4)
    merge5 = concatenate([conv2, up5], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    up6 = UpSampling2D()(conv5)
    merge6 = concatenate([conv1, up6], axis=3)
    conv6 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    output = Conv2D(3, (3, 3), activation='softmax', padding='same')(conv6)

    model = Model(inputs=inputs, outputs=output)
    return model

# Create and summarize the model
model = create_HRnet_model()
model.summary()


# Compile && Train

In [ ]:
reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1
)

e_decay = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.98,
)

f1_score = F1Score(name='f1_score')


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), f1_score])

history = model.fit(train_set,
                    validation_data = test_set,
                    epochs=100,
                    callbacks=[
                        reduce_lr_callback
                    ]
                   )

# Predict

In [ ]:
i = 30
image , label =train_set.take(1).get_single_element()
print(image.shape)
plt.imshow(image[i].numpy().astype("uint8"))

In [ ]:
pred = model.predict(np.expand_dims(image[i], axis=0))
plt.imshow(pred[0])
print(pred.shape)